![EBAC](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/ebac_logo-data_science.png?raw=true)

# **Projeto Final do Curso de SQL da EBAC**
### **Exploração e análise de dados de crédito com SQL**

**Aluno:** [Marcelo Aires ![LinkedIn](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/linkedin.png?raw=true)](https://www.linkedin.com/in/marceloaco/)

---

## **1. Dados do Dataset:** 

Os dados representam informações de clientes de um banco e contam com as seguintes colunas: 

| *Coluna*                  | *Explicação*                                            |
|---------------------------|---------------------------------------------------------|
| **idade**                 | idade do cliente                                        |
| **sexo**                  | sexo do cliente (F ou M)                                |
| **dependentes**           | número de dependentes do cliente                        |
| **escolaridade**          | nível de escolaridade do cliente                        |
| **salario_anual**         | faixa salarial do cliente                               |
| **tipo_cartao**           | tipo de cartão do cliente                               |
| **qtd_produtos**          | quantidade de produtos comprados nos últimos 12 meses   |
| **iteracoes_12m**         | quantidade de iterações/transações nos últimos 12 meses |
| **meses_inativo_12m**     | quantidade de meses que o cliente ficou inativo         |
| **limite_credito**        | limite de crédito do cliente                            |
| **valor_transacoes_12m**  | valor das transações dos últimos 12 meses               |
| **qtd_transacoes_12m**    | quantidade de transações dos últimos 12 meses           |


A tabela foi criada no **AWS Athena** junto com o **S3 Bucket**, a própria EBAC disponibilizou o arquivo CSV.

---

## **2. Criação da Tabela no AWS Athena:**

Após criar o bucket no AWS S3 e carregar o CSV disponibilizado pela EBAC, podemos criar a Tabela no AWS Athena vinculado com o bucket que iremos utilizar neste projeto.

Para criarmos a tabela utilizamos este comando:

#### CREATE TABLE:

```sql
CREATE EXTERNAL TABLE IF NOT EXISTS default.credito (
	`idade` int,
	`sexo` string,
	`dependentes` int,
	`escolaridade` string,
	`estado_civil` string,
	`salario_anual` string,
	`tipo_cartao` string,
	`qtd_produtos` bigint,
	`iteracoes_12m` int,
	`meses_inativo_12m` int,
	`limite_credito` float,
	`valor_transacoes_12m` float,
	`qtd_transacoes_12m` int
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
	'serialization.format' = ',',
	'field.delim' = ','
)
LOCATION 's3://credito-projetofinal-marceloaco/'
TBLPROPERTIES ('has_encrypted_data' = 'false');
```
![Query1](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/Criar-Tabela.png?raw=true)

---

## **3. Análise Exploratória de Dados (EDA):**

Após a criação da tabela no *AWS*, podemos analisar o conjunto de dados pelo *Athena*.

#### **3.1. Número de Linhas:**
```sql
SELECT COUNT(*) AS total_linhas
FROM credito;
```
![totalLinhas](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/totalLinhas.png?raw=true)

#### **3.2. Número de Colunas:**
```sql
SELECT COUNT(*) AS total_colunas
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'credito';
```
![totalColunas](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/totalColunas.png?raw=true)

---

#### **3.3. Tipos de cada coluna:**
```C#
DESCRIBE credito;
```
| Coluna               | Tipo   |
| -------------------- | ------ |
| idade                | int    |
| sexo                 | string |
| dependentes          | int    |
| escolaridade         | string |
| estado_civil         | string |
| salario_anual        | string |
| tipo_cartao          | string |
| qtd_produtos         | bigint |
| iteracoes_12m        | int    |
| meses_inativo_12m    | int    |
| limite_credito       | float  |
| valor_transacoes_12m | float  |
| qtd_transacoes_12m   | int    |

---

#### 3.4. QUERY 1:

```sql
SELECT * FROM credito LIMIT 10;
```
![Limite10](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/credito_lim10.png?raw=true)

> Primeira visualização da Tabela (10 primeiras linhas), já percebemos alguns valores nulos.

---

#### 3.5. QUERY 2:

```sql
SELECT sexo, COUNT(sexo) AS Quantidade FROM credito GROUP BY sexo;
```

![sexoQuant](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/sexoQuant.png?raw=true)

``` sql
SELECT escolaridade, COUNT(escolaridade) AS Quantidade FROM credito GROUP BY escolaridade;
```

![escolaridadeNA](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/escolaridadeNA.png?raw=true)

``` sql
SELECT estado_civil, COUNT(estado_civil) AS Quantidade FROM credito GROUP BY estado_civil;
```

![estadoCivilNA](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/estadoCivilNA.png?raw=true)

``` sql
SELECT salario_anual, COUNT(salario_anual) AS Quantidade FROM credito GROUP BY salario_anual;
```

![salarioAnualNA](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/salarioAnualNA.png?raw=true)

``` sql
SELECT tipo_cartao, COUNT(tipo_cartao) AS Quantidade FROM credito GROUP BY tipo_cartao;
```

![tiposCartaoQuant](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/tiposCartaoQuant.png?raw=true)


> Analisando cada uma das colunas, percebemos aonde estão os valores nulos, como mostra nas figuras acima.

> Percebemos que na coluna 'sexo' contem mais Homens do que Mulheres. 

> E na coluna 'tipo_cartao' verificamos que existem 4 tipos de cartões, sendo o cartão blue mais comum entre eles.
---

## **4. Análise de Dados:**

Após a exploração dos dados e do entedimento do nosso dataset, podemos analisar os dados e buscar aprofundar ainda mais no entendimento do negócio.

#### **4.1 Média do Limite de Crédito em relação ao salário anual:**
```sql
SELECT AVG(limite_credito) AS media_limite_credito,
	salario_anual
FROM credito
WHERE salario_anual != 'na'
GROUP BY salario_anual
ORDER BY media_limite_credito DESC;
```
![mediaLimiteCreditoSalario](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/mediaLimiteCreditoSalario.png?raw=true)

> Vemos que existe um padrão para liberação de crédito de acordo com o salário anual do cliente.

---

#### **4.2. Análise de sexo:**
```sql
SELECT sexo,
	COUNT(*) AS quantidade
FROM credito
GROUP BY sexo;
```
![sexoQuant](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/sexoQuant.png?raw=true)
![graficoSexoQuant](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/graficoSexoQuant.png?raw=true)

> Vemos que o número de homens é superior ao número de mulheres. Vamos nos aprofundar nas etapas seguintes.

#### **4.2.1. Média do Limite de Crédito em relação ao Sexo:**

```sql
SELECT sexo,
	AVG(limite_credito) AS Media_Limite_Credito,
	COUNT(sexo) AS Quantidade
FROM credito
GROUP BY sexo
ORDER BY Quantidade DESC;
```

![mediaLimiteCreditoSexo](https://github.com/marceloaires7/EBAC-SQL/blob/ba9ea1c1001a9855cc804e68f5d21f0c8e150def/MODULO_08/images/mediaLimiteCreditoSexo.png?raw=true)
![grafMediaLimiteCreditoSexo](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/grafMediaLimiteCreditoSexo.png?raw=true)

> Observa-se uma grande discrepância entre os créditos liberados para homens e mulheres.

#### **4.2.2. Salário Anual em relação ao Sexo:**

```sql
SELECT sexo,
	salario_anual,
	COUNT(sexo) AS Quantidade
FROM credito
WHERE salario_anual != 'na'
GROUP BY sexo, salario_anual
ORDER BY Quantidade DESC;
```

![salarioAnualSexoQuant](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/salarioAnualSexoQuant.png?raw=true)
![graficoSalarioSexoQuant](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/graficoSalarioSexoQuant.png?raw=true)

> Observamos que a grande discrepância na consulta anterior se deve à significativa desproporção entre homens e mulheres no que diz respeito ao salário anual. O dataset fornecido pela EBAC, infelizmente, contém apenas informações de mulheres com renda baixa.

---

#### **4.3. Análise de Escolaridade:**
```sql
SELECT escolaridade,
	COUNT(*) AS Quantidade
FROM credito
WHERE escolaridade != 'na'
GROUP BY escolaridade
ORDER BY Quantidade DESC;
```
![escolaridadeQuant](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/escolaridadeQuant.png?raw=true)
![graficoEscolaridadeQuant](https://github.com/marceloaires7/EBAC-SQL/blob/29691defefa366b771c212e08ff9cb72c4b545c1/MODULO_08/images/graficoEscolaridadeQuant.png?raw=true)

> Existem mais clientes com 'mestrado', seguidos por 'ensino medio', 'sem educacao formal', 'graduacao' e 'doutorado'.

#### **4.3.1. Média do Limite de Crédito em relação a Escolaridade:**

```sql
SELECT AVG(limite_credito) AS Media_Limite_Credito,
	escolaridade
FROM credito
WHERE escolaridade != 'na'
GROUP BY escolaridade
ORDER BY Media_Limite_Credito DESC
```

![mediaEscolaridadeLimiteCredito](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/mediaEscolaridadeLimiteCredito.png?raw=true)
![graficoEscolaridadeLimiteCredito](https://github.com/marceloaires7/EBAC-SQL/blob/main/MODULO_08/images/graficoEscolaridadeLimiteCredito.png?raw=true)

> A média dos limites de crédito não parece alterar de acordo com a escolaridade, todos tem um valor aproximado.

---

## **5. Conclusão:**

Com este projeto podemos retirar algumas análises extraídas do dataset.

- O DataSet é formado por 2564 linhas e 13 colunas.
- Das 13 colunas, 3 apresentam dados nulos ('escolaridade', 'estado_civil', 'salario_anual').
- O Limite de Crédito aumenta de acordo com o Salário Anual do Cliente.
- O DataSet apresenta mais clientes Homens do que Mulheres.
- O DataSet apresenta inconsistência em relação a coluna 'sexo', em que só foram inseridas mulheres de Renda Baixa.
- Existem mais clientes com 'mestrado', seguidos por 'ensino medio', 'sem educacao formal', 'graduacao' e 'doutorado'.

Seria mais prudente buscar um dataset que apresente informações mais abrangentes e equilibradas. Poderíamos considerar realizar uma nova busca por dados ou procurar o dataset completo, focando em variáveis que proporcionem uma visão mais equitativa da situação.